In [159]:
import pandas as pd
import os # package for using operating system
from dotenv import load_dotenv
from snowflake.snowpark.session import Session # package for building and using Snowflake sessions
# from snowflake.cortex import Complete
from transformers import AutoTokenizer # package to select the fitting tokenizer for a pretrained model
from huggingface_hub import login # package for login and identifying to Huggingface
import pandas as pd # package for data manipulation
import json # package to work with .json
import math # package for mathematical operations

https://arxiv.org/pdf/2303.04048 --> reference-free & reference-based metric (direct-assessment or one-to-five stars)  
https://arxiv.org/pdf/2407.07959 --> llm evaluation


In [160]:
# with open('openstack_openstack.json', 'r') as f:
#     loaded_data = json.load(f)

In [161]:
with open('openstack_openstack_output.json', 'r') as f:
    loaded_data = json.load(f)

In [162]:
input_txt = loaded_data['readme']
repo_name = loaded_data['repo_owner']

In [163]:
print(input_txt)

## OpenStack Git Submodule Manager

## Description
The OpenStack Git Submodule Manager is a Python script designed to manage Git submodules for OpenStack projects. It fetches a list of integrated gate repositories, generates a `.gitmodules` file, and configures the Git submodules accordingly.

## Installation
To install the OpenStack Git Submodule Manager, you will need to have Python and Git installed on your system. You will also need to install the `requests` and `yaml` libraries using pip:
```bash
pip install requests yaml
```
## Usage
To use the script, simply run it from the command line:
```bash
python openstack_git_submodule_manager.py
```
This will fetch the list of integrated gate repositories, generate the `.gitmodules` file, and configure the Git submodules.

## Contributing
Contributions to the OpenStack Git Submodule Manager are welcome! If you'd like to contribute, please fork the repository and submit a pull request. Make sure to follow the standard OpenStack contributi

In [164]:
repo_name

'openstack'

In [165]:
prompt_evaluation = f'''
You are acting as a software development expert for the following GitHub repository "{repo_name}".
Your task is to evaluate the following README of the repository/project based on the provided questions:
    - q1: What is the goal of the project?
    - q2: Why is the project useful?
    - q3: How can users get started with the project?
    - q4: Where can users get help with your project?
    - q5: Who maintains and contributes to the project?
How well does the provided README offers answers to each question? Score each question on a scale from 1 to 5, 
in which 1 means "insufficient", 2 "sufficienct", 3 "satisfactory", 4 "good" and 5 "very good". Provide your chosen score with a short explanation.
A good indication of a high score is: 1) there is a section or paragraph where ab answer of the questions is clearly provided,
2) the content of the README is clearly structured and understandable 3) that a developer who reads the README understands the purpose of the project.
You're not allowed to skip a question or add any small talk!
Structure your response in the following format:
###
"q1": [
    ##"score": your_score##,
    ##"explanation": your_explanation##
    ]
###
"q2": ...
###
Here is the README to evaluate:
{input_txt}
'''

In [166]:
# alle noten definieren!
## oder von jeder "Note" eines ansehen.
## oder besonders schlechte und besonders gute readmes selbst ansehen (ich als experte ... haha) und (ein-zwei random aus der mitte
## oder große delta ansehen.(z.b. schlecht erzeugtes readme zu gutem originalem readme)
## plot von hecht kategorien auftragen (meines gegen das originale)

In [167]:
model = 'llama3.1-8b'
# specify llm parameters for summary creation
model_params = {
   'temperature': 0, # default: 0 https://docs.snowflake.com/en/sql-reference/functions/complete-snowflake-cortex --> Internetrecherche hat keine anderen Empfehlungen ergeben
   # 'top_p': # default: 0 https://docs.snowflake.com/en/sql-reference/functions/complete-snowflake-cortex
    'max_tokens': 400
}

In [168]:
# load .env file
load_dotenv(override=True)

# set up connection parameters for Snowflake connection
connection_params = {
    "account": os.environ['SNOWFLAKE_ACCOUNT'],
    "user": os.environ['SNOWFLAKE_USER'],
    "password": os.environ['SNOWFLAKE_USER_PASSWORD'],
    "role": 'SYSADMIN',
    "warehouse": 'COMPUTE_WH',
    'paramstyle': 'qmark'
}

# build Snowflake session with connection parameters
snowflake_session = Session.builder.configs(connection_params).create()
print('Snowflake sessions is build.')
print('---------------------------------------------')

Snowflake sessions is build.
---------------------------------------------


In [169]:
query = f"""
            SELECT SNOWFLAKE.CORTEX.COMPLETE( 
                ?,
                [
                    {{
                        'role': 'user', 
                        'content': ?
                    }}
                ],
                {{
                    'temperature': ?,
                    'max_tokens':  ?
                }} 
            ) AS response
        """
response = snowflake_session.sql(query, params=[model, prompt_evaluation, model_params['temperature'], model_params['max_tokens']]).collect()
res = json.loads(response[0]['RESPONSE']) # load response as json object and save it in 'res'
# split 'res' parts and save them into multiple variabels
message = res['choices'][0]['messages'] 

In [170]:
print(message)
# scores für original

### "q1": [
    ##"score": 4##,
    ##"explanation": The README clearly states the purpose of the project, which is to manage Git submodules for OpenStack projects. However, it would be even better if the description was more detailed and explained the benefits of using the script."##
]

### "q2": [
    ##"score": 3##,
    ##"explanation": The README mentions that the project is useful for managing Git submodules for OpenStack projects, but it does not provide any specific examples or use cases of how the project can be useful. A more detailed explanation of the benefits and value proposition of the project would make it more convincing."##
]

### "q3": [
    ##"score": 5##,
    ##"explanation": The README provides a clear and concise step-by-step guide on how to get started with the project, including installation and usage instructions. The instructions are easy to follow and provide a good first impression of the project."##
]

### "q4": [
    ##"score": 2##,
    ##"explanation": Th

In [199]:
snowflake_session.close()

In [173]:
import re

In [182]:
text = score[0]
text = '[\n    ##"score": 46##,\n '
pattern = r'\d##'
matches = re.findall(pattern, text)
matches

['6##']

In [176]:
score[0]

'"q1": [\n    ##"score": 4##,\n    ##"explanation": The README clearly states the purpose of the project, which is to manage Git submodules for OpenStack projects. However, it would be even better if the description was more detailed and explained the benefits of using the script."##\n]\n\n'

In [200]:
score_txt = message
#score_txt = score_txt.replace("'", "\'")
score_list = score_txt.split('### ')
score = score_list[1:]
print(score)
print('-----------------')
score_dir = []
pattern_digit = r"\d##"
#pattern_explanation = r""

for i in score:
    line = i.split('\n ')
    print(line)
    tmp_dir = {
        'question': line[0].split(':')[0].replace('"', ''),
        #'score': int(line[1].split(': ')[1].replace(':', '').replace(',', '')),
        'score': int(re.findall(pattern_digit, line[1])[0].replace('#','')),
        'explanation': line[2].split('#"explanation": ')[1].replace('"', '').replace('\n', '').replace('\n\n', '')
    }

    score_dir.append(tmp_dir)



['"q1": [\n    ##"score": 4##,\n    ##"explanation": The README clearly states the purpose of the project, which is to manage Git submodules for OpenStack projects. However, it would be even better if the description was more detailed and explained the benefits of using the script."##\n]\n\n', '"q2": [\n    ##"score": 3##,\n    ##"explanation": The README mentions that the project is useful for managing Git submodules for OpenStack projects, but it does not provide any specific examples or use cases of how the project can be useful. A more detailed explanation of the benefits and value proposition of the project would make it more convincing."##\n]\n\n', '"q3": [\n    ##"score": 5##,\n    ##"explanation": The README provides a clear and concise step-by-step guide on how to get started with the project, including installation and usage instructions. The instructions are easy to follow and provide a good first impression of the project."##\n]\n\n', '"q4": [\n    ##"score": 2##,\n    ##"e

In [201]:
score_dir

[{'question': 'q1',
  'score': 4,
  'explanation': 'The README clearly states the purpose of the project, which is to manage Git submodules for OpenStack projects. However, it would be even better if the description was more detailed and explained the benefits of using the script.##]'},
 {'question': 'q2',
  'score': 3,
  'explanation': 'The README mentions that the project is useful for managing Git submodules for OpenStack projects, but it does not provide any specific examples or use cases of how the project can be useful. A more detailed explanation of the benefits and value proposition of the project would make it more convincing.##]'},
 {'question': 'q3',
  'score': 5,
  'explanation': 'The README provides a clear and concise step-by-step guide on how to get started with the project, including installation and usage instructions. The instructions are easy to follow and provide a good first impression of the project.##]'},
 {'question': 'q4',
  'score': 2,
  'explanation': "The RE

In [78]:
score_txt = message

In [124]:
import json

In [194]:
#path = '../data/output_evaluation_data/model1/Taniiishk_Rock-Paper-Scissors-Game_evaluation_output.json'
path = '../data/output_evaluation_data/model1/openstack_openstack_evaluation_output.json'

In [195]:
with open(path, 'r') as f: 
    loaded_data = json.load(f)

In [196]:
test_score = loaded_data['readme_original']['evaluation']

In [197]:
test_score

'### "q1": \n[\n    "score": 2,\n    "explanation": The goal of the project is not clearly stated in the provided README. The first paragraph mentions that OpenStack is a collection of interoperable components, but it does not explicitly state the main objective of the project. The second paragraph explains the purpose of this specific repository, which is to provide a read-only view of the tested combinations of OpenStack projects, but it is not the main goal of the OpenStack project as a whole.\n]\n\n### "q2": \n[\n    "score": 4,\n    "explanation": The project is useful because it provides a read-only view of the tested combinations of OpenStack projects, which can be useful for users who want to understand how the projects interact with each other. The README also mentions that this feature is important for some users, but it does not elaborate on the specific use cases or benefits.\n]\n\n### "q3": \n[\n    "score": 2,\n    "explanation": The README provides a link to the OpenStac

In [198]:
score_txt = test_score
#score_txt = score_txt.replace("'", "\'")
score_list = score_txt.split('### ')
score = score_list[1:]
print(score)
print('-----------------')
score_dir = []
pattern_digit = r"\d##"
#pattern_explanation = r""

for i in score:
    line = i.split('\n ')
    print(line)
    tmp_dir = {
        'question': line[0].split(':')[0].replace('"', ''),
        #'score': int(line[1].split(': ')[1].replace(':', '').replace(',', '')),
        'score': re.findall(pattern_digit, line[1])[0].replace('#',''),
        'explanation': line[2].split('#"explanation": ')[1].replace('"', '').replace('\n', '').replace('\n\n', '')
    }

    score_dir.append(tmp_dir)

['"q1": \n[\n    "score": 2,\n    "explanation": The goal of the project is not clearly stated in the provided README. The first paragraph mentions that OpenStack is a collection of interoperable components, but it does not explicitly state the main objective of the project. The second paragraph explains the purpose of this specific repository, which is to provide a read-only view of the tested combinations of OpenStack projects, but it is not the main goal of the OpenStack project as a whole.\n]\n\n', '"q2": \n[\n    "score": 4,\n    "explanation": The project is useful because it provides a read-only view of the tested combinations of OpenStack projects, which can be useful for users who want to understand how the projects interact with each other. The README also mentions that this feature is important for some users, but it does not elaborate on the specific use cases or benefits.\n]\n\n', '"q3": \n[\n    "score": 2,\n    "explanation": The README provides a link to the OpenStack s

IndexError: list index out of range

In [144]:
score[1]

'"q2": \n[\n    "score": 4,\n    "explanation": The project is useful because it provides a read-only view of the tested combinations of OpenStack projects, which can be useful for users who want to understand how the projects interact with each other. The README also mentions that this feature is important for some users, but it does not elaborate on the specific use cases or benefits.\n]\n\n'